In [ ]:
# add autoreload
%load_ext autoreload
%autoreload 2
import os
import sys
import json

import numpy as np
import pandas as pd
import scipy as sc

from collections import defaultdict
import re
import gc
import deduce
import random

from tqdm import tqdm
import seaborn as sns
from typing import Callable

from gensim.models import phrases

import matplotlib.pyplot as plt

import dill

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import medcat_utils as mu

# Context:
* $100$ K echocardiographic reports available. 
* we want to extract diagnoses regarding the left-ventricle function
* we have $5000$ reports with labeled spans.

# Goal:
Train a "model" that can
1. identify the spans
2. classify the spans

# Approach: MedCAT - MetaCAT

## Two-step approach

* unsupervised training on the documents
* add a single custom entity with a custom identifier
* train a model to identify the custom entities
* supervised training on the meta-annotations of the entities

## One-step approach

* unsupervised training on the documents
* add custom entities based on the spans and their labels
* train a model to identify the custom entities

# Approach: biLSTM/transformer

## Two-step approach

* Train a model to identify the spans: self-supervision by random selecting non-span ranges as negative examples
* Train a model to classify the spans: supervised based on the labeled spans 
* Combine the model in one pipeline

## One-step approach
* Assign a label to each span
* Train a model to identify the spans

In [ ]:
# TODO:..
# 1. by adding a "no_label" class to each label, in memory
# 2. re-train with reduced labels
# 3. re-evaluate with whole pipeline!

##  Load Medcat modelpack

In [ ]:
# load dotenv
from dotenv import load_dotenv
load_dotenv('../.env')
# extract the environment variable 'medcat_pack'


In [ ]:
from medcat.cat import CAT
from medcat.vocab import Vocab
from medcat.cdb import CDB
from medcat.config import Config
from medcat.meta_cat import MetaCAT

In [ ]:
base_medcat_path = os.getenv('medcat_pack')
pack_location = 'umls-dutch-v1-10_echo'
prep_medcat = False
train_with_negatives = True
REDUCED=False
PRETRAIN_META=False 

# Load texts

In [ ]:
echo_path = 'T://lab_research/RES-Folder-UPOD/Echo_label/E_ResearchData/2_ResearchData'
# load the jsonl in a dataframe
texts = pd.read_json(os.path.join(echo_path, 'outdb_140423.jsonl'), lines=True)
#texts_zipped = zip(texts['text'], texts['_input_hash'], texts['_task_hash'])

# Load train/test splits

In [ ]:
train_indcs = pd.read_csv(os.path.join(echo_path, 'train_echoid.csv'), sep=',')
test_indcs = pd.read_csv(os.path.join(echo_path, 'test_echoid.csv'), sep=',')

train_indcs = train_indcs[train_indcs.input_hash.notna()]
test_indcs = test_indcs[test_indcs.input_hash.notna()]

In [ ]:
TRAIN_INPUT_HASH = set(train_indcs['input_hash'].astype(int).to_list())
TRAIN_TASK_HASH = set(train_indcs['task_hash'].astype(int).to_list())

TEST_INPUT_HASH = set(test_indcs['input_hash'].astype(int).to_list())
TEST_TASK_HASH = set(test_indcs['task_hash'].astype(int).to_list())

In [ ]:
texts_train = texts.loc[texts._input_hash.isin(TRAIN_INPUT_HASH)]
texts_test = texts.loc[texts._input_hash.isin(TEST_INPUT_HASH)]

# Load MedCAT model

In [ ]:
MCAT = CAT.load_model_pack(str(os.path.join(base_medcat_path, pack_location)))

## Make splitted data

* MedCAT train files
* class-dictionary with dataframes

In [ ]:
# psyheartanalyse
# psyheartanalyse baseline

# TODO:
# lv_syst_func: 
# lv_sys_func_unchanged, lv_sys_func_improved -> lv_sys_func_unknown
# lv_syst_func_normal -> lv_sys_func_normal

# rv_syst_func:
# rv_syst_func_normal -> rv_sys_func_normal

In [ ]:
if REDUCED==True:
    file_dir = os.path.join(echo_path, 'echo_span_labels', 'reduced_labels')
    class_names = os.listdir(file_dir)
else:
    file_dir = os.path.join(echo_path, 'echo_span_labels', 'full_labels')
    class_names = os.listdir(file_dir)
    

In [ ]:
merged_index = [i for i,t in enumerate(class_names) 
                    if ('merged' in t) | ('old' in t)]
merged_index = sorted(merged_index, reverse=True)
merged_name = class_names[merged_index[0]]

for mind in merged_index:
    class_names.pop(mind)

In [ ]:
class_ds = defaultdict(dict)

for file_name in class_names:   
    _class = file_name.split(".")[0]
    fn = os.path.join(file_dir, file_name)
    class_ds[_class]['ds'] = pd.read_json(fn, lines=True)
    
    class_ds[_class]['train'] = {}
    class_ds[_class]['test'] = {}    
    
    medcat_json = mu.get_medcat_json_per_class(fn, TRAIN_INPUT_HASH, _class)
    if train_with_negatives:
        medcat_json = mu.update_medcat_json_per_class_with_negatives(MedCATJSON=medcat_json, ClassName=_class)
        
    medcat_out = os.path.join(echo_path, 'medcat_labels', 'train', f'medcat_{_class}.json')
    json.dump(medcat_json, open(medcat_out, 'w'))
    class_ds[_class]['train']['location'] = medcat_out
    class_ds[_class]['train']['json'] = medcat_json
     
    medcat_json = mu.get_medcat_json_per_class(fn, TEST_INPUT_HASH, _class)
    if train_with_negatives:
        medcat_json = mu.update_medcat_json_per_class_with_negatives(MedCATJSON=medcat_json, ClassName=_class)  
    
    medcat_out = os.path.join(echo_path, 'medcat_labels', 'test', f'medcat_{_class}.json')
    json.dump(medcat_json, open(medcat_out, 'w'))
    class_ds[_class]['test']['location'] = medcat_out
    class_ds[_class]['test']['json'] = medcat_json
    
    class_ds[_class]['labels'] = set([span_dict['label'] 
                                        for span_list in class_ds[_class]['ds']['spans'].tolist()
                                        for span_dict in span_list])
    if train_with_negatives:
        class_ds[_class]['labels'].add('nolabel')
        # we want to collect per inputhash per span
        # if there is any label
        # if there is a label for the current class
        spans_per_doc = class_ds[_class]['ds'][['_input_hash', 'spans']]\
                            .set_index('_input_hash')\
                            .to_dict(orient='index')

        span_tuples_per_doc = {k:[(_v['start'], _v['end']) 
                                  for _v in v['spans']] 
                                  for k,v in spans_per_doc.items()
                                  }
        class_ds[_class]['spans_with_labels'] = span_tuples_per_doc
        

In [ ]:
if train_with_negatives:
    SpansPresent = defaultdict(set)
    SpanClassPresent = defaultdict(lambda: defaultdict(set))
    for k,v in class_ds.items():
        for inp_hash, sl in class_ds[k]['spans_with_labels'].items():
            for _sl in sl:
                SpansPresent[inp_hash].add(_sl)
                SpanClassPresent[k][inp_hash].add(_sl)

In [ ]:
ClassMap = defaultdict(dict)
for k, v in class_ds.items():
    for lab in v['labels']:
        ClassMap[lab] = k
ClassMap['normal'] = 'normal'

## Unsupervised learning for NER+L

In [ ]:
if PRETRAIN_META:
    MCAT.train(texts_train.text.values, 
                nepochs=3, 
                progress_print=10,  
                is_resumed=True)
    MCAT.create_model_pack(base_medcat_path + "/umls-dutch-v1-10_echo")

### Add label spans from Prodigy annotations

In [ ]:
span_sets = defaultdict(set)
for _class, dsd in tqdm(class_ds.items()):
        ds = dsd['ds']
        span_set = set()
        span_list = []
        ds = ds[ds._input_hash.isin(TRAIN_INPUT_HASH)]
        for k, (_spans, text) in enumerate(zip(ds[ds.spans.notna()].spans.values,
                                            ds[ds.spans.notna()].text.values)):
            for _span in _spans:
                start, end = _span['start'], _span['end']
                span_set.add(text[start:end])
                span_list.append(text[start:end])
        span_sets[_class] = span_set

In [ ]:
# TODO: interesting to add variants without the abbreviations?
# TODO: including paraphrasing?

In [ ]:
if PRETRAIN_META:
    for _class, span_set in tqdm(span_sets.items()):
        for _span in span_set:
            MCAT.add_and_train_concept(cui=_class,
                                    name=_span, 
                                    do_add_concept=True,
                                    negative=False,
                                )
            
    MCAT.train_supervised(data_path=os.path.join(base_medcat_path, 
                                     "umls-dutch-v1-10_echo",
                                     "input/ner_l_anno/trainer_export.json"), 
                          nepochs=4,
                          print_stats=0,
                          use_filters=False)
    medcat_path = base_medcat_path + "/umls-dutch-v1-10_echoV2"
    MCAT.create_model_pack(medcat_path)

## Supervised learning of MetaCAT models

In [ ]:
# load MCAT from file
medcat_path = os.path.join(base_medcat_path, 'umls-dutch-v1-10_echoV2')

In [ ]:
MCAT = CAT.load_model_pack(medcat_path)

In [ ]:
from medcat.meta_cat import MetaCAT
from medcat.config_meta_cat import ConfigMetaCAT
from medcat.tokenizers.meta_cat_tokenizers import TokenizerWrapperBPE, ByteLevelBPETokenizer

# TODO: train embedding based on training text

In [ ]:
# load tokenizer from negation_model
# tokenizer folder 
tok_folder = 'T:/laupodteam/AIOS/Bram/language_modeling/Clinical_embeddings/unigrams/with_tokenizer/v2/tokenizer'
emb_folder = 'T:/laupodteam/AIOS/Bram/language_modeling/Clinical_embeddings/unigrams/with_tokenizer/v2/SG'
tokenizer = ByteLevelBPETokenizer.from_file(os.path.join(tok_folder, 'vocab.json'), 
                                            os.path.join(tok_folder, 'merges.txt'))
wrapped_tokenizer = TokenizerWrapperBPE(hf_tokenizers=tokenizer)
wrapped_tokenizer.save(medcat_path + "/assets/tokenizer")

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
vec_path = os.path.join(emb_folder, 'sg')
print(vec_path)
w2v = KeyedVectors.load(vec_path)

In [ ]:
# Create embedding matrix
embeddings = []
words_not_present = []

in_vocab = 0
for i in range(tokenizer.get_vocab_size()):
    word = tokenizer.id_to_token(i)
    if word in w2v:
        embeddings.append(w2v[word])
        in_vocab +=1
    else:
        words_not_present.append(i)
        embeddings.append(np.random.random(300))

print(f"In vocab:{in_vocab}")
mean_vector = np.mean(embeddings, axis=0)

for i in words_not_present:
    embeddings[i] = mean_vector

# Save the embeddings
embeddings_array = np.array(embeddings)
np.save(open(medcat_path+"/assets/embeddings/embedding.npy", 
             'wb'), embeddings_array)

print(f"Words not present:{len(words_not_present)}")

In [ ]:
for _class, d in class_ds.items():
    if _class != 'normal':
        print(f"Commencing training of biLSTM-span for {_class}...")
        config_metacat = ConfigMetaCAT()
        config_metacat.general['category_name'] = _class
        config_metacat.train['nepochs'] = 25
        config_metacat.train['score_average'] = 'macro'
        config_metacat.model['hidden_size'] = 16
        config_metacat.model['input_size'] = 300
        config_metacat.model['dropout'] = 0.25
        config_metacat.model['num_layers'] = 32
        config_metacat.model['num_directions'] = 2
        config_metacat.model['nclasses'] = len(d['labels'])
        config_metacat.model['model_name'] = 'lstm'
        
        meta_cat = MetaCAT(tokenizer=wrapped_tokenizer,
                           embeddings=embeddings_array,
                           config=config_metacat)
        
        train_path = d['train']['location']
        model_path = os.path.join(medcat_path, f"meta_{_class}")
        os.makedirs(model_path, exist_ok=True)
        
        meta_cat.train_from_json(json_path=train_path, 
                                 save_dir_path=model_path)
        
        meta_cat.save(save_dir_path=model_path)
        # now manually add the model to the model_pack...
        label_dict = config_metacat.general.category_value2id
        
        # add to config
        medcat_config = json.load(open(os.path.join(medcat_path, 'model_card.json'), 'r'))
        
        # add to "MetaCAT models" list
        medcat_config["MetaCAT models"].append({
                      "Category Name": _class,
                      "Description": f"Labels: {REDUCED}, Negatives: {train_with_negatives}",
                      "Classes": label_dict,
                        "Model": "lstm"
                    })
        # write config to .json
        gc.collect()
        json.dump(medcat_config, open(os.path.join(medcat_path, 'model_card.json'), 'w'))

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
print(f"learnable parameters: {count_parameters(meta_cat.model)}")

## Load  new model pack

In [ ]:
os.path.join(medcat_path)

In [ ]:
MCATnew = CAT.load_model_pack(os.path.join(medcat_path))

## Apply to texts

In [ ]:
from spacy import displacy

In [ ]:
i = 1123
doc = MCATnew(texts.text.values[i])
displacy.render(doc, style='ent')

In [ ]:
doc = MCATnew(texts.text.values[i])
inds = []
res = []
for ent in doc.ents:
    inds.append(ent.text)
    res.append(mu.create_dict_with_conf(ent))
    
res_df = pd.DataFrame(res, index=inds)

## Evaluate

In [ ]:
def get_test_results(medcat_path:str=None, class_ds: dict=None, score_average: str="macro"):
    TEST_RESULTS = {}
    for k in class_ds.keys():
        if k != 'normal':
            CATmodel = MetaCAT.load(os.path.join(medcat_path, f'meta_{k}'))
            CATmodel.config['train']['score_average'] = score_average # weighted, macro
            test_location = class_ds[k]['test']['location']
            test_labels = json.load(open(test_location, 'r'))
            bulk_res = CATmodel.eval(test_location)
            
            res_count = defaultdict(int)
            for doc in test_labels['projects'][0]['documents']:
                anns = doc['annotations']
                if len(anns)>0:
                    for ann in anns:            
                        _ann = ann['meta_anns'][k]
                        res_count[_ann['value']] += 1
    
            TEST_RESULTS[k] = {
                'f1':  bulk_res['f1'],
                'precision': bulk_res['precision'],
                'recall': bulk_res['recall'],
                'confusion_df': bulk_res['confusion matrix'],
                'real_presence': res_count
            }
    return TEST_RESULTS

In [ ]:
TEST_RESULTS_MACRO = get_test_results(medcat_path=medcat_path, class_ds=class_ds, score_average='macro')
TEST_RESULTS_WEIGHTED = get_test_results(medcat_path=medcat_path, class_ds=class_ds, score_average='weighted')

In [ ]:
res_dict_list = []
rnd_nmbr = 2
for k,v in TEST_RESULTS_WEIGHTED.items():
    res_dict_list.append({
            'class': k,
            'f1': str(round(TEST_RESULTS_WEIGHTED[k]['f1'],rnd_nmbr))+"("+str(round(TEST_RESULTS_MACRO[k]['f1'],rnd_nmbr))+")",
            'recall': str(round(TEST_RESULTS_WEIGHTED[k]['recall'],rnd_nmbr))+"("+str(round(TEST_RESULTS_MACRO[k]['recall'],rnd_nmbr))+")",
            'precision': str(round(TEST_RESULTS_WEIGHTED[k]['precision'],rnd_nmbr))+"("+str(round(TEST_RESULTS_MACRO[k]['precision'],rnd_nmbr))+")",
        })
res_df = pd.DataFrame(res_dict_list)

In [ ]:
dill.dump(res_df, file=open("../artifacts/MetaCAT_test_results_with_negatives_reduced.pkl", "wb"))
#TEST_RESULTS = dill.load(open("../artifacts/MetaCAT_test_results.pkl", "rb"))

## Assessment of performance in the wild

In [ ]:
# refactor this code for seperate json file per class
merged_labels_train = mu.get_medcat_json_per_class(open(os.path.join(file_dir, 'merged_labels.jsonl'), 'r'), 
                                    TRAIN_INPUT_HASH, 'merged', ClassMap=ClassMap)['projects'][0]['documents']

merged_labels_test = mu.get_medcat_json_per_class(open(os.path.join(file_dir, 'merged_labels.jsonl'), 'r'), 
                                    TEST_INPUT_HASH, 'merged', ClassMap=ClassMap)['projects'][0]['documents']

In [ ]:
comparison_list = []
for medcat_doc in tqdm(merged_labels_test):
    txt = medcat_doc['text']
    parsed_doc = MCATnew(txt)

    res = []
    start_stop = []
    for ent in parsed_doc.ents:
        start_stop.append((ent.start_char, ent.end_char))
        res.append(mu.create_dict_with_conf(ent))
    
    medcat_doc['predicted'] = {k:v for k,v in zip(start_stop, res)}
    comparison_list.append(medcat_doc)
comparison_list_with_annotations_present = [d for d in comparison_list if len(d['annotations'])>0]

In [ ]:
def get_pred_list(pred_dict: dict=None):
    return list(set([v for k,v in pred_dict.items() if 'conf_' not in k]))
    
span_suggester_comparison_list = []
for d in tqdm(comparison_list_with_annotations_present):
    anns = d['annotations']
    span_cat_list = []
    for ann in anns:
        span = (ann['start'], ann['end'])
        class_val = ann['meta_anns']['merged']['value']
        span_cat_list.append((span, class_val))
    
    pred_list = []
    for k, pred in d['predicted'].items():
        pred_list.append((k, get_pred_list(pred)))
    
    span_suggester_comparison_list.append([span_cat_list, pred_list])

In [ ]:
# We want to get the coverage of spans -> a count of all (partially) overlapping spans
# We want to get the token overlap of covered spans (Jaccard?)
# span overlap count

def _tuple_overlap(tL, tR):
    # tL: tuple(begin, end)
    # tR: tuple(begin, end)
    tLrange = set(range(*tL))
    tRrange = set(range(*tR))
               
    InterSection = len(tLrange.intersection(tRrange))
    Union = len(tLrange.union(tRrange))
    
    return InterSection/Union if Union>0 else np.nan

def span_overlap_counter(labeled_spans, reversed=False):
    # label_spans: list[[list[((begin,end), label)], list[((begin,end), [labels])]]]
    OverlapList = []
    for span_labs in labeled_spans:
        span_set_medcat = set()
        span_set_labeled = set()
        for lab_span in span_labs[0]:
            span_set_labeled.add(lab_span[0])
        
        for med_span in span_labs[1]:
            span_set_medcat.add(med_span[0])
        
        # check overlap
        jaccard_indices = []
        if reversed:
            left_spans = span_set_medcat
            right_spans = span_set_labeled          
        else:
            left_spans = span_set_labeled
            right_spans = span_set_medcat
        
        for spanL in left_spans:
            _jaccard_indices = []            
            for spanR in right_spans:
                _jaccard_indices.append(_tuple_overlap(spanL, spanR))
            try:
                max_ = max(_jaccard_indices)
            except:
                max_ = np.nan
            jaccard_indices.append(max_)
        OverlapList.append(jaccard_indices)
    return OverlapList

def span_overlap_counter_with_assignment(labeled_spans):
    # label_spans: list[[list[((begin,end), label)], list[((begin,end), [labels])]]]
    OverlapList = []
    for span_labs in labeled_spans:
        span_set_medcat = set()
        span_dict_labeled = defaultdict(set)
        for lab_span in span_labs[0]:
            span_dict_labeled[lab_span[1]].add(lab_span[0])
        
        for med_span in span_labs[1]:
            span_set_medcat.add(med_span[0])
        
        # check overlap
        jaccard_indices = defaultdict(list)
        for spanClass, spanLabs in span_dict_labeled.items():
            for spanLab in spanLabs:                         
                _jaccard_indices = []   
                for spanMedcat in span_set_medcat:
                    _jaccard_indices.append(_tuple_overlap(spanLab, spanMedcat))
                try:
                    max_ = max(_jaccard_indices)
                except:
                    max_ = np.nan
                jaccard_indices[spanClass].append(max_)
        OverlapList.append(jaccard_indices)
    return OverlapList
    

In [ ]:
ignore_list = ['normal', 'lv_syst_func_normal', 'nolabel', 'lv_sys_func_unknown', 'lv_sys_func_unchanged', 'lv_sys_func_improved', 'pe']
acceptableLabels = [_label for _label in ClassMap.keys() if _label not in ignore_list]

def accepted_labels(labels, accepted_labels):
    return any([(_label in accepted_labels) for _label in labels])

In [ ]:
ClassMap.pop('normal')
ClassMap.pop('pe')
ClassMap.pop('lv_sys_func_improved')
ClassMap.pop('lv_sys_func_unchanged')
ClassMap.pop('lv_syst_func_normal')

In [ ]:
# end-to-end performance, requires an implicit "no label" label for consistency
# assuming one-versus-all for scoring
# tp: present and pos pred & pos lab
# tn: present and neg pred & neg lab (i.e. not a particular class value)
# fp: present and pos pred & neg lab (i.e. not a particular class value)
# fn: present and neg pred & neg lab 
# The only difference with the exact matching is that we add a negative label called "no label"

'''
ConfusionDict = {'Class': {
    'ClassValue1': {
        'fp': 
        'fn': 
        'tp': 
        'tn': 
    } 
}}
'''
# from true -> predicted 
##########################
min_jacc = 0.5 # This is somewhat arbitrary: caveat it is :)
totList = []
for comp in span_suggester_comparison_list:
    docDict = defaultdict(lambda: defaultdict(lambda : defaultdict(int)))
    conc_true_list = defaultdict(list)
    for (tTrue_l, tTrue_r), True_class_value in comp[0]:
        conc_true_list[(tTrue_l, tTrue_r)].append(True_class_value)
        
    for tTrue, True_labs in conc_true_list.items():
        # check if there is an overlapping span (with some minimum Jaccard)
        _classes = [ClassMap.get(True_lab) for True_lab in True_labs] 
        pred_checked = []  # to avoid double counting false positives
        for i, _c in enumerate(_classes):
            found = False
            if (_c is not None) & (isinstance(_c, str)):        
                for tPred, Pred_labs in comp[1]:
                    # TODO: check if any of the labels is in accepted list of labels
                    if accepted_labels(Pred_labs, acceptableLabels):
                        if _tuple_overlap(tTrue, tPred)>min_jacc:                        
                            if True_labs[i] in Pred_labs:
                                docDict[_c][True_labs[i]]['tp'] += 1
                                # fp for all other class values in Pred_labs?
                                for m, pred_lab in enumerate(Pred_labs):
                                   if m not in pred_checked: 
                                       if (pred_lab not in True_labs) and (pred_lab not in ['nolabel', 
                                                                                            'negated', 
                                                                                            'not negated',
                                                                                            'lv_syst_func_normal',
                                                                                            'lv_sys_func_improved',
                                                                                            'lv_sys_func_unchanged'
                                                                                            ]):
                                           __c = ClassMap.get(pred_lab)
                                           pred_checked.append(m)
                                           if (__c is not None) & (isinstance(__c, str)):
                                               docDict[__c][pred_lab]['fp'] += 1 
                                found = True
                            else:
                                docDict[_c][True_labs[i]]['fn'] += 1
                if not found:
                    docDict[_c][True_labs[i]]['fn'] += 1
    totList.append(docDict)
    
# collect per class per class value the amount of tp, fp, fn
finalDict = defaultdict(lambda: defaultdict(lambda : defaultdict(int)))
for d in totList:
    for k,v in d.items():
        for _k, _v in v.items():
            for __k, __v in _v.items():
                finalDict[k][_k][__k] += __v

In [ ]:
# span_suggester_comparison_list
# make dictionary with the count of predicted spans per class and the number of false prediction per class

ClassTpCount = defaultdict(int)
ClassPredCount = defaultdict(int)
TotSpans = 0
min_jacc = 0.25
for SpanLists in span_suggester_comparison_list:
    # [((l,r), label)]
    TrueSpans = SpanLists[0]
    # [((l,r), [list of labels])]
    PredictedSpans = SpanLists[1]
    
    for PredictedSpan in PredictedSpans:
        TotSpans += 1 
        predSpan = PredictedSpan[0]
        predLabels = PredictedSpan[1]
        for predLabel in predLabels:
            if predLabel in acceptableLabels:
                ClassPredCount[ClassMap[predLabel]] += 1
        
        for TrueSpan in TrueSpans:
            trueSpan = TrueSpan[0]
            trueLabel = TrueSpan[1]
            if trueLabel in acceptableLabels:
                if _tuple_overlap(trueSpan, predSpan)>min_jacc:
                    if trueLabel in predLabels:
                        ClassTpCount[ClassMap[trueLabel]] += 1
    

In [ ]:
ClassFpRate = defaultdict(float)
ClassFpRateC = defaultdict(float)
for k,v in ClassPredCount.items():
    ClassFpRate[k] = round((v-ClassTpCount[k])/v, 2)
    ClassFpRateC[k] = round((v-ClassTpCount[k])/TotSpans, 2)

In [ ]:
res_list = []
for k,v in ClassFpRate.items():
    res_list.append({'class': k, 
                     'fprate': str(ClassFpRateC[k])+"("+str(v)+")"})
pd.DataFrame(res_list).sort_values(by='class').set_index('class')


In [ ]:
# given the tp, fp, fn the precision, recall, f1 can be calculated
eps = 1e-7
precision = lambda tp,fp: tp/(tp+fp+eps)
recall = lambda tp,fn: tp/(tp+fn+eps)
f1 = lambda p,r: 2*p*r/(p+r+eps)

In [ ]:
scoreDict = defaultdict(lambda: defaultdict(lambda : defaultdict(int)))
for k,v in finalDict.items():
    for _k, _v in v.items():
        fp = _v['fp'] if not isinstance(_v['fp'], dict) else 0
        fn = _v['fn'] if not isinstance(_v['fn'], dict) else 0
        tp = _v['tp'] if not isinstance(_v['tp'], dict) else 0
        totCount = fp + fn + tp        
        prec = precision(tp,fp)
        rec = recall(tp,fn)
        scoreDict[k][_k]['count'] = totCount
        scoreDict[k][_k]['precision'] = prec
        scoreDict[k][_k]['recall'] = rec
        scoreDict[k][_k]['f1'] = f1(prec,rec)

In [ ]:
classScores = defaultdict(lambda: defaultdict(float))
for k,v in scoreDict.items():
    totCount = 0
    precF, precM = 0,0
    recF, recM = 0,0
    f1F, f1M = 0,0
    for i, _v in enumerate(v.values()):
        totCount += _v['count']
        precF += _v['precision']*_v['count']
        recF += _v['recall']*_v['count']
        f1F += _v['f1']*_v['count']
        
        precM += _v['precision']
        recM += _v['recall']
        f1M += _v['f1']
        
    classScores[k]['precision'] = str(round(precF/totCount,2)) + "("+str(round(precM/(i+1),2))+")"
    classScores[k]['recall'] = str(round(recF/totCount,2)) + "("+str(round(recM/(i+1),2))+")"
    classScores[k]['f1'] = str(round(f1F/totCount,2)) + "("+str(round(f1M/(i+1),2))+")"
    
    

In [ ]:
pd.DataFrame(classScores).round(2).T.sort_index(ascending=True)

In [ ]:
OverlapJaccardIndices = span_overlap_counter(span_suggester_comparison_list, reversed=False)
OverlapJaccardIndicesReversed = span_overlap_counter(span_suggester_comparison_list, reversed=True)
OverlapJaccardClassIndices = span_overlap_counter_with_assignment(span_suggester_comparison_list)

In [ ]:
PerClassValueJaccard = defaultdict(list)
for d in OverlapJaccardClassIndices:
    for k,v in d.items():
        PerClassValueJaccard[k].extend(v)

In [ ]:
PerClassJaccard = defaultdict(list)
failed_cvs = set()
for cv, v in PerClassValueJaccard.items():
    try:
        PerClassJaccard[ClassMap[cv]].extend(v)
    except:
        failed_cvs.add(cv)

In [ ]:
MeanPerDocument = [np.nanmean(v) for v in OverlapJaccardIndices]
MeanOverall = np.nanmean([_v for v in OverlapJaccardIndices for _v in v])

MeanPerDocumentRev = [np.nanmean(v) for v in OverlapJaccardIndicesReversed]
MeanOverallRev = np.nanmean([_v for v in OverlapJaccardIndicesReversed for _v in v])


print(f"Mean document coverage: {np.mean(MeanPerDocument)}, with {MeanOverall} over all tokens")

plt.hist(MeanPerDocument, bins=50, color='green', histtype='step', lw=2);
plt.hist(MeanPerDocumentRev, bins=50, color='red', histtype='step', lw=2);

plt.title("Span overlap per document")

In [ ]:
[{'class': k, 'mean': np.nanmean(v)} for k,v in PerClassJaccard.items()]

In [ ]:
MinMean = 0.25
CheckRes = [{'Jaccard': v, 
             'res': comparison_list_with_annotations_present[i]} 
            for i,v in enumerate(OverlapJaccardIndices)
            if (np.mean(v)<MinMean) | (np.isnan(np.mean(v)))]

In [ ]:
print(f"We have {len(CheckRes)} documents to check")

In [ ]:
CheckRes[0]['res']['predicted']

# Document classification from span classification

In [ ]:
AllowedClasses = set(ClassMap.values())

In [ ]:
SeverityMap = {
    '_mild':2,
    '_moderate':3,
    '_severe':4,
    '_not_present': 0,
    '_unknown': 1,
    '_normal': 0,
    '_present': 1,
    'nolabel': -1
}
SeverityMapFull = {}
for lab in acceptableLabels:
    for k, score in SeverityMap.items():
        if k in lab:
            SeverityMapFull[lab] = score
            break
SeverityMapFull['nolabel'] = -1

In [ ]:
trueClass = []
predClass = []
# [{'input_hash': xx, 'aortic_regurgitation': f_worst(aortic_regurgitation_labels}...]

def get_worst_labels(LabelDict: dict=None, ScoreMap: dict=None):
    # per class get worst labels, if no labels present for class return 'nolabel' for this class
    OutDict = {_class: "nolabel" for _class in AllowedClasses}
    
    for k,v in LabelDict.items():
        worst=-1
        for label in v:
            score = ScoreMap[label]
            if score>worst:
                worst = score
                OutDict[k] = label
    return OutDict
    
for d in comparison_list:
    r = {}
    p = {}
    ld = defaultdict(set)
    r['input_hash'] = d['input_hash']
    p['input_hash'] = d['input_hash']
    
    for ann in d['annotations']:
        ClassName = ann['meta_anns']['merged']['name']
        LabelName = ann['meta_anns']['merged']['value']
        if isinstance(ClassName, str):
            if (ClassName in AllowedClasses) and (LabelName in acceptableLabels):
                ld[ClassName].add(LabelName)
        else:
            print(f"True ClassName is wrong type {ClassName}")
    lw = get_worst_labels(ld, SeverityMapFull)  
    r.update(lw)
    trueClass.append(r)
    
    rd = defaultdict(set)
    for _, predDict in d['predicted'].items():
        for _class, pred in predDict.items():
            if (_class in AllowedClasses) and (pred in acceptableLabels):
                rd[_class].add(pred)
    pw = get_worst_labels(rd, SeverityMapFull)
    p.update(pw)
    predClass.append(p)
    


In [ ]:
predClassDF = pd.DataFrame(predClass).set_index('input_hash')
trueClassDF = pd.DataFrame(trueClass).set_index('input_hash')

In [ ]:
df_dict = {
    c: predClassDF[[c]].join(trueClassDF[[c]],lsuffix='_pred', rsuffix='_true')
    for c in predClassDF.columns
}

In [ ]:
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score, 
                             confusion_matrix, roc_auc_score, multilabel_confusion_matrix)

In [ ]:
df_dict.keys()

In [ ]:
res_list = []
for c in df_dict.keys():
    ypred = df_dict[c].iloc[:,0]
    ytrue = df_dict[c].iloc[:,1]
    f1_macro = f1_score(ytrue,ypred, average='macro')
    f1_weighted = f1_score(ytrue,ypred, average='weighted')
    recall_macro = recall_score(ytrue,ypred, average='macro')
    recall_weighted = recall_score(ytrue,ypred, average='weighted')
    precision_macro = precision_score(ytrue,ypred, average='macro')
    precision_weighted = precision_score(ytrue,ypred, average='weighted')
    
    res_list.append({
        'class': c,
        'f1': str(round(f1_weighted,2)) + "(" + str(round(f1_macro,2)) + ")",
        'recall': str(round(recall_weighted,2)) + "(" + str(round(recall_macro,2)) + ")",
        'precision': str(round(precision_weighted,2)) + "(" + str(round(precision_macro,2)) + ")",
    })

In [ ]:
pd.DataFrame(res_list).sort_values(by='class')
